In [8]:

import ee
import geetools
import folium
import time
from folium import plugins
ee.Authenticate()
ee.Initialize()
import Leaf_tools

Enter verification code:  4/1AY0e-g7uRi_BRliQy5B4UjNHhEEQOewIavLwJuacnKTmLy5onbLUt4cdv4U



Successfully saved authorization token.


In [9]:
ezone = ee.FeatureCollection("users/ganghong/NA_Forests")

In [10]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)

def renameB1(image): 
   return image.select("b1").rename("partition")

def imgUnmask(image): 
   return image.unmask()

def imgValid(image): 
    score = image.select('QC')
    return score.eq(ee.Number(0)) 

def imgOutofrange(image): 
    score = image.select('QC')    
    return  score.eq(ee.Number(2)) or (score.eq(ee.Number(3)))

def imgOutofdomain(image): 
    score = image.select('QC')
    return  score.eq(ee.Number(1)) or (score.eq(ee.Number(3)))  

def newDoYBandforClearSky(image): 
    qc=image.select("QC")
    mask = qc.mask()
    doy = qc.date().getRelative('day', 'year')
    return ee.Image.constant(doy).int().clip(image.unmask().geometry()).updateMask(mask).copyProperties(image, ["system:time_start"])

def newDoYBandforValidRetrieval(image):
    qc=ee.Image(image.select("QC"))
    maskvalid= qc.eq(ee.Number(0))##qc.not()  ## mask invalid pixels, keep valid pixels (corresponds to 0), not() function turns all zeros to 1, all others to 0 assume the same meaning in Colab
    #maskvalid= qc.Not()
    doy = qc.date().getRelative('day', 'year')
    return ee.Image.constant(doy).int().clip(image.unmask().geometry()).updateMask(maskvalid).copyProperties(image, ["system:time_start"])

basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}

def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))

def statsbasedonregion(Belmanip_bio, intersectarea):
    #######Task 6:Average elapsed days between observations//////////////////////////////////////// passed
    """Task7: Average elapsed days between clear sky """
    clearSkydays=ee.ImageCollection(Belmanip_bio).map(newDoYBandforClearSky)   
    reducers = ee.Reducer.minMax().combine(
                    reducer2=ee.Reducer.count(),
                    sharedInputs=True
                )
    clearSkydaysStat= clearSkydays.reduce(reducers)
    clear_maxband=clearSkydaysStat.select(1) ##max
    clear_minband=clearSkydaysStat.select(0) ##min
    clear_countband=clearSkydaysStat.select(2) ##count 
    clear_interval=clear_maxband.subtract(clear_minband).divide(clear_countband.subtract(ee.Number(1))).rename('clear_interval') ## (max-min)/(count-1)
    
    mean_clear_interval = clear_interval.reduceRegion(
      **{'reducer': ee.Reducer.mean(),
      'geometry': intersectarea.geometry(),
      'scale': 400,
      'maxPixels': 1e9
      })

    ## The result is a Dictionary.  Print it.
    print(mean_clear_interval.getInfo(),'mean_clear_interval_ezone')


    ## Reduce the region to forest area. 
    mean_clear_interval_forest = clear_interval.updateMask(NALCMS_forest).reduceRegion(
      **{'reducer': ee.Reducer.mean(),
      'geometry': intersectarea.geometry(),
      'scale': 400,
      'maxPixels': 1e9
      })

    ## The result is a Dictionary.  Print it.
    print(mean_clear_interval_forest.getInfo(),'mean_clear_interval_forest')

    """Taks 8: Average elapsed days between valid retrievals"""   
    validDays=ee.ImageCollection(Belmanip_bio).map(newDoYBandforValidRetrieval)
    validDaysStat= validDays.reduce(reducers)
    valid_maxband=validDaysStat.select(1)##  //max
    valid_minband=validDaysStat.select(0) ##//min
    valid_countband=validDaysStat.select(2) ## //count
    valid_interval=valid_maxband.subtract(valid_minband).divide(valid_countband.subtract(ee.Number(1))).rename('valid_interval') ##  // (max-min)/(count-1)   

    ## Reduce to ezone 
    mean_valid_interval = valid_interval.reduceRegion(
        **{
      'reducer': ee.Reducer.mean(),
      'geometry': intersectarea.geometry(),
      'scale': 400,
      'maxPixels': 1e9
    })

    ## The result is a Dictionary.  Print it.
    print(mean_valid_interval.getInfo(),'mean_valid_interval_ezone')

    ## Reduce to forest 
    mean_valid_interval_forest = valid_interval.updateMask(NALCMS_forest).reduceRegion(
        **{
      'reducer': ee.Reducer.mean(),
      'geometry': intersectarea.geometry(),
      'scale': 400,
      'maxPixels': 1e9
    })
    ## The result is a Dictionary.  Print it.
    print(mean_valid_interval_forest.getInfo(),'mean_valid_interval_forest')

def polygonintersect(feat1):    
    feat1 = ee.Feature(feat1.geometry());   
    intersection = feat1.intersection(feat2, ee.ErrorMargin(1));
    return intersection.set({'Intersect': intersection.area().divide(1000 * 1000)})
   
def polyInterFeatureCollection(polygon, featurecollection):
    #polygons2=Belmanip_Site1_LAI.first().unmask().geometry()
    feat2 = ee.Feature(polygon)
    intersects =ezone.map(polygonintersect)   
    #intersects =ezone.map(lambda feat1:polygonintersect(*feat1))
    sortedintersect=intersects.sort('Intersect',false)
    return sortedinterset.first()


In [12]:
### calculate the intersection area between ezone and image
tiles = ee.FeatureCollection('users/ganghong/TilesSampling_Belmanip_Sub')
tiles = tiles.map(lambda feat: feat.buffer(-10000))
#print (table.size().getInfo())
ezonepolygon=ee.FeatureCollection(ezone)
for i in range (0, tiles.size().getInfo()):
    mapBounds = ee.FeatureCollection(ee.Feature(tiles.toList(tiles.size()).get(i))).geometry()
    ## get tile number
    tile=ee.Feature(tiles.toList(tiles.size()).get(i)).get('MGRS_TILE')
    print (tile.getInfo(),'TileNumber')
    ## forest land cover
    NALCMS_forest = ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles').map(renameB1)\
    .filterBounds(mapBounds)\
    .mosaic()\
    .clip(mapBounds)\
    .remap([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],[ 1, 1, 1, 1, 1, 1, 1, 1, 0,  0,  0,  0, 0, 1, 0, 0, 0, 0, 0],0,"partition")\
    .rename('ForestCover')
    
    ##get image intersection with dominant ezone
    feat2 = ee.Feature(mapBounds)    
    intersects=ezonepolygon.map(polygonintersect)
    intersectstemp=ee.FeatureCollection(intersects).sort('Intersect',False)
    intersectarea=intersectstemp.first()## dominant ezone intersect with image
    ## region statiscal analysis for 2019 and 2020
    print ("2019")
     ## run leaf tool box for 2019 
    Belmanip_2019=Leaf_tools.Leaf_tools(mapBounds, 2019)
    print (Belmanip_2019.size().getInfo(), "2019 acquisitions")
    statsbasedonregion(Belmanip_2019, intersectarea)
    print("2020")
     ## run leaf tool box for  2020
    Belmanip_2020=Leaf_tools.Leaf_tools(mapBounds, 2020)
    print (Belmanip_2020.size().getInfo(), "2020 acquisitions")
    statsbasedonregion(Belmanip_2020, intersectarea)
    

15SVV TileNumber
2019
47 2019 acquisitions
{'clear_interval': 6.697903618802091} mean_clear_interval_ezone
{'clear_interval': 6.697134667834354} mean_clear_interval_forest
{'valid_interval': 6.7453744834034985} mean_valid_interval_ezone
{'valid_interval': 6.744398607748596} mean_valid_interval_forest
2020
44 2020 acquisitions
{'clear_interval': 6.657578578274759} mean_clear_interval_ezone
{'clear_interval': 6.64795920515511} mean_clear_interval_forest
{'valid_interval': 6.699078576010707} mean_valid_interval_ezone
{'valid_interval': 6.690096134061548} mean_valid_interval_forest
15SWR TileNumber
2019
53 2019 acquisitions
{'clear_interval': 7.333893739930358} mean_clear_interval_ezone
{'clear_interval': 7.5591386927513815} mean_clear_interval_forest
{'valid_interval': 7.389194445757925} mean_valid_interval_ezone
{'valid_interval': 7.614504584738026} mean_valid_interval_forest
2020
48 2020 acquisitions
{'clear_interval': 8.304420966402077} mean_clear_interval_ezone
{'clear_interval': 8.60